In [ ]:
#import packages


In [2]:
#system packages

import sys
import warnings
import os 
import traceback #obs? 
if not sys.warnoptions:
    warnings.filterwarnings("once")  

In [3]:
#base packages:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats, integrate, optimize
import math
import datetime
from datetime import date, timedelta

In [4]:
#optional packages

from statsmodels.tsa.base.datetools import dates_from_str
from IPython.display import display
pd.options.display.max_columns = 50  #allow DF.head to show all columns in notebook
from see import see
from tabulate import tabulate 
#from io import StringIO

In [5]:
#packages for the econometrics / models

from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import DFGLS, ADF, KPSS, PhillipsPerron, ZivotAndrews
from arch.unitroot.cointegration import engle_granger, phillips_ouliaris
import statsmodels.formula.api as smf  #VAR package contained within 
import statsmodels.tsa.api as smt

#import the functionality for detecting mathematical errors (E.G. types of linear algebra issues etc.)
from statsmodels.tools.sm_exceptions import ValueWarning
from arch.utility.exceptions import (
    InfeasibleTestException,
    InvalidLengthWarning,
    invalid_length_doc)
warnings.filterwarnings("once", category = ValueWarning)

C:\Users\JasonSpano\anaconda3\lib\site-packages\patsy\constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [6]:
#sorted_alldata_df_final = pd.read_pickle("./sorted_alldata_df_final.pkl")
sorted_alldata_df_final = pd.read_pickle(r"C:\Users\JasonSpano\OneDrive\Uva Msc 19-20 Subjects\Research Assistant Docs\CDS_project_-_first_set_of_tasks\Part 2_jan20./sorted_alldata_df_final.pkl")

# ADF: (#2)
* https://www.statsmodels.org/stable/examples/notebooks/generated/stationarity_detrending_adf_kpss.html

In [19]:
sorted_alldata_df_final

,gvkey,trade_date,bond_sym_id,price,issue_id,bond_in_mergent,coupon,interest_frequency,coupon_type,maturity_date,secured,seniority,years_to_maturity,riskfree_rate_1yr,riskfree_rate_2yr,riskfree_rate_3yr,riskfree_rate_4yr,riskfree_rate_5yr,market_cds_spread,series_id,initial_price,solved_lambda,year_index,pecds,cds_bond_basis,year_month,sector,cboe_vix,ted_rate,ticker,company_name,trading_vol_daily,price_close,price_high,price_low,price_open,eps,price_sales,price_cashflow,gross_profit_ta,capital_ratio,debt_eq_ratio,rnd_sales
0,1045,2009-09-24,AMR.UY,103.1100,504701.0,True,6.25,2.0,F,20141015.0,NaN,1.0,5.056914,0.580,1.240,1.860,2.350,2.710,1468.95,AMR5$AX,12.129838,0.036953,2009,12.046157,1456.903843,2009-09-01,Transporation,24.95,0.18,AAMRQ,AMERICAN AIRLINES GROUP INC,34493080.0,7.74,8.03,7.63,7.91,-3.51,0.071,-0.949,0.134,1.288,-13.427,0.000
1,1045,2009-09-25,AMR.UY,107.0000,504701.0,True,6.25,2.0,F,20141015.0,NaN,1.0,5.054176,0.620,1.290,1.900,2.370,2.710,1493.47,AMR5$AX,12.123144,0.037014,2009,12.066573,1481.403427,2009-09-01,Transporation,25.61,0.18,AAMRQ,AMERICAN AIRLINES GROUP INC,31763720.0,8.02,8.14,7.90,7.94,-3.51,0.071,-0.949,0.134,1.288,-13.427,0.000
2,1045,2009-09-28,AMR.UY,104.2500,504701.0,True,6.25,2.0,F,20141015.0,NaN,1.0,5.045963,0.620,1.300,1.890,2.340,2.660,1522.60,AMR5$AX,12.126682,0.037318,2009,12.167274,1510.432726,2009-09-01,Transporation,24.88,0.17,AAMRQ,AMERICAN AIRLINES GROUP INC,13330220.0,8.00,8.20,7.90,8.06,-3.51,0.071,-0.949,0.134,1.288,-13.427,0.000
3,1045,2009-09-29,AMR.UY,105.7500,504701.0,True,6.25,2.0,F,20141015.0,NaN,1.0,5.043225,0.640,1.330,1.900,2.340,2.660,1535.88,AMR5$AX,12.124804,0.037335,2009,12.172879,1523.707121,2009-09-01,Transporation,25.19,0.16,AAMRQ,AMERICAN AIRLINES GROUP INC,16459960.0,8.00,8.25,7.98,8.06,-3.51,0.071,-0.949,0.134,1.288,-13.427,0.000
4,1045,2009-09-30,AMR.UY,105.6268,504701.0,True,6.25,2.0,F,20141015.0,NaN,1.0,5.040487,0.630,1.300,1.880,2.310,2.640,1634.44,AMR5$AX,12.130400,0.037417,2009,12.200371,1622.239629,2009-09-01,Transporation,25.61,0.15,AAMRQ,AMERICAN AIRLINES GROUP INC,18732570.0,7.95,8.12,7.71,8.02,-3.51,0.121,-1.623,0.134,1.288,-13.427,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460267,179700,2017-06-26,DPS.AI,103.1190,564989.0,True,3.20,2.0,F,20211115.0,0.0,1.0,4.388865,1.421,1.546,1.651,1.738,1.815,230.31,DPS5$AX,-5.925957,0.162585,2017,110.343059,119.966941,2017-06-01,Consumer Staples,9.90,0.31,DPS,DR PEPPER SNAPPLE GROUP INC,1103392.0,92.92,93.76,92.68,93.26,4.57,2.640,19.860,0.431,0.633,3.289,0.003
460268,179700,2017-06-27,DPS.AI,103.5360,564989.0,True,3.20,2.0,F,20211115.0,0.0,1.0,4.386127,1.436,1.573,1.688,1.783,1.868,230.31,DPS5$AX,-5.922706,0.162339,2017,110.161852,120.148148,2017-06-01,Consumer Staples,11.06,0.31,DPS,DR PEPPER SNAPPLE GROUP INC,895203.0,91.63,92.70,91.39,92.62,4.57,2.640,19.860,0.431,0.633,3.289,0.003
460269,179700,2017-06-28,DPS.AI,103.6400,564989.0,True,3.20,2.0,F,20211115.0,0.0,1.0,4.383389,1.434,1.571,1.690,1.790,1.877,230.31,DPS5$AX,-5.922515,0.162290,2017,110.125823,120.184177,2017-06-01,Consumer Staples,10.03,0.30,DPS,DR PEPPER SNAPPLE GROUP INC,1197095.0,92.22,92.79,91.82,92.03,4.57,2.640,19.860,0.431,0.633,3.289,0.003
460270,179700,2017-06-29,DPS.AI,103.4370,564989.0,True,3.20,2.0,F,20211115.0,0.0,1.0,4.380651,1.448,1.598,1.724,1.832,1.925,230.31,DPS5$AX,-5.919429,0.162070,2017,109.963876,120.346124,2017-06-01,Consumer Staples,11.44,0.28,DPS,DR PEPPER SNAPPLE GROUP INC,853952.0,90.90,92.02,90.70,91.98,4.57,2.640,19.860,0.431,0.633,3.289,0.003


In [20]:
gvkey_frame = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==179700]

In [25]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [26]:
adf_test(gvkey_frame['market_cds_spread'])

Results of Dickey-Fuller Test:
Test Statistic                   -5.387622
p-value                           0.000004
#Lags Used                       20.000000
Number of Observations Used    1477.000000
Critical Value (1%)              -3.434785
Critical Value (5%)              -2.863499
Critical Value (10%)             -2.567813
dtype: float64


In [27]:
adfuller(gvkey_frame['market_cds_spread'], maxlag=None, 
         regression='c',
         autolag='AIC',
         store=True, 
         regresults=True)

(-5.387622148488619,
 3.5959786595425608e-06,
 {'1%': -3.434785139702456,
  '5%': -2.863498825305098,
  '10%': -2.5678128583805213},
 <statsmodels.tsa.stattools.ResultsStore at 0x26a8695a108>)

# ADF TEST EXPLANATION: 
* Single GVKEY

In [ ]:
gvkey_selection = 179700
gvkey_frame = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==gvkey_selection]

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
vix = gvkey_frame['cboe_vix']

gvkey_frame_adf = pd.DataFrame(data=(cds, pecds, vix)).T
var_model_adf = smt.VAR(gvkey_frame_adf)  
#res_model_adf = var_model_adf.fit(maxlags=var_model_adf.select_order(maxlags=5).bic)  #VAR MODEL
bic_selected_lag_adf = var_model_adf.select_order(maxlags=5).bic

ADF_cds = ADF(cds, lags=bic_selected_lag_adf)
ADF_cds.summary()

In [ ]:
gvkey_frame.sector.iloc[0]

In [ ]:
ADF(pecds, lags=bic_selected_lag_adf).summary()

In [ ]:

#select KEY
gvkey_1408 = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==1408]

gvkey_number = 1408
gvkey_frame = gvkey_1408

print("JOHANSENS COINT TEST with individual GVKEY NUMBER: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
data = pd.DataFrame(data=(cds, pecds)).T
#data = np.column_stack((cds, pecds))

model = vecm.select_order(data, maxlags=5)
optimal_lag_length = model.bic
    
vecm_cointeg_rank = vecm.select_coint_rank(data, det_order =1, k_ar_diff = optimal_lag_length, 
                                                  method='trace', signif=0.05)
        
print(vecm_cointeg_rank.summary())
print("Number of Cointegrating relationships:", vecm_cointeg_rank.rank)

# VAR REGRESSION

In [ ]:
#VAR MODEL WITH GRAPH

gvkey_1045 = sorted_alldata_df_final.loc[sorted_alldata_df_final.gvkey==1045]

In [ ]:
gvkey_1045['price_close'].plot();

In [ ]:
gvkey_1045['cboe_vix'].plot();

In [ ]:
gvkey_frame = gvkey_1045

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
stock_close = gvkey_frame['price_close'].pct_change().dropna()
vix = gvkey_frame['cboe_vix']
#vix = gvkey_frame['cboe_vix'].pct_change().dropna()

#[1:] == skip the first data point to align the series' lengths
gvkey_frame_4 = pd.DataFrame(data=(cds[1:], pecds[1:], vix[1:], stock_close)).T

var_model_4 = smt.VAR(gvkey_frame_4)  #add make lags here? 

res_model_4 = var_model_4.fit(maxlags=var_model_4.select_order(maxlags=5).bic)

#print(res_model_4)
print("max lags:", var_model_4.select_order(maxlags=5).bic,'\n')
print(res_model_4.summary())

In [ ]:
#NEED TO ADD DIVIDEND BACK 

#PG 14 of 81 in LEE & ETAL:
#try model-adjusted stock returns: >?



In [ ]:
#VAR model 

df_gvkey_150937 = df_basis_coupons_month.loc[df_basis_coupons_month.gvkey==150937.0]

#specify individual series
cds_150937 = df_gvkey_150937['market_cds_spread']
pecds_150937 = df_gvkey_150937['pecds']      

#alternatively, some methods require that a DF be used to maintain column labels/references
gvkey_150937_solved = pd.DataFrame(data=(cds_150937, pecds_150937)).T

#VAR_MODEL: GVKEY==2136
var_model_gvkey_150937 = smt.VAR(gvkey_150937_solved)
res_model_gvkey_150937 = var_model_gvkey_150937.fit(maxlags=var_model_gvkey_150937.select_order(maxlags=5).bic)
print("max_lags:", var_model_gvkey_150937.select_order().bic)
print(res_model_gvkey_150937.summary())

In [ ]:
#Original VECM base: CDS/PECDS

gvkey_number = 1408
gvkey_frame = gvkey_1408
#coint_rels = 0

print("VECM MODEL OF GVKEY: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
vecm_data_base = pd.DataFrame(data=(cds, pecds)).T

vecm_model_base = vecm.select_order(vecm_data_base, maxlags=5)
optimal_lag_length_base = vecm_model_base.bic
    
vecm_cointeg_rank_base = vecm.select_coint_rank(vecm_data_base, det_order =1, k_ar_diff = optimal_lag_length_base, 
                                                  method='trace', signif=0.05)

if vecm_cointeg_rank_base.rank >= 1:
    coint_rels_base = 1
else:
    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_base = vecm.VECM(vecm_data_base, k_ar_diff=vecm_model_base.bic, coint_rank=coint_rels_base, deterministic='cili')
vecm_base_res = vecm_base.fit()
print(vecm_base_res.summary())




vecm_base_res.test_granger_causality(caused=0).summary()

In [ ]:
#VECM of base + VIX: CDS/PECDS
#with automated coint rels calculation

gvkey_number = 1408
gvkey_frame = gvkey_1408
#coint_rels = 0

print("VECM MODEL + VIX OF GVKEY: " + str(gvkey_number))

cds = gvkey_frame['market_cds_spread']
pecds = gvkey_frame['pecds']
vix = gvkey_frame['cboe_vix']
vecm_data_vix = pd.DataFrame(data=(cds, pecds, vix)).T

vecm_model_vix = vecm.select_order(vecm_data_vix, maxlags=5)
optimal_lag_length_vix = vecm_model_vix.bic
    
vecm_cointeg_rank_vix = vecm.select_coint_rank(vecm_data_vix, det_order =1, k_ar_diff = optimal_lag_length_vix, 
                                                  method='trace', signif=0.05)

#if vecm_cointeg_rank_vix.rank >= 1:
#    coint_rels_vix = 1
#else:
#    print("insufficient number of cointegrating relationships for VECM model")
    
vecm_vix_2 = vecm.VECM(vecm_data_vix, k_ar_diff=vecm_model_vix.bic, coint_rank=vecm_cointeg_rank_vix.rank, deterministic='cili')
vecm_vix_res_2 = vecm_vix_2.fit()
print(vecm_vix_res_2.summary())



vecm_vix_res_2.test_granger_causality(causing='cboe_vix', caused='market_cds_spread').summary()